In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [ ]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


In [ ]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

In [ ]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

In [ ]:
print(column_names)

In [ ]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [ ]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)
            

In [ ]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [1]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

In [2]:
import pandas as pd
url = 'https://raw.githubusercontent.com/Melissa795/SpaceX-Falcon-9-Prediction-Landing/main/dataset_part_1.csv'
data = pd.read_csv(url)
 

In [3]:
 
# Filtrar los lanzamientos que no son Falcon 1
data_falcon9 = data[data['BoosterVersion'] != 'Falcon 1']
# Reiniciar la numeración de los vuelos
data_falcon9.loc[:, 'FlightNumber'] = list(range(1, len(data_falcon9) + 1))
# Mostrar cuántos lanzamientos de Falcon 9 hay
print(len(data_falcon9))

90


In [4]:
 
print(data['BoosterVersion'].value_counts())


Falcon 9    90
Name: BoosterVersion, dtype: int64


In [5]:
 
# Mostrar cuántos lanzamientos hay por tipo de cohete
print(data['BoosterVersion'].value_counts())
 


Falcon 9    90
Name: BoosterVersion, dtype: int64


In [ ]:
data.isnull().sum()

In [ ]:
 
# Ver cuántos valores faltan en la columna 'landingPad'
faltantes_landingpad = data['landingPad'].isnull().sum()
print(f"Valores faltantes en 'landingPad': {faltantes_landingpad}")

In [8]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

In [10]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

In [12]:
print(data.columns)

Index(['FlightNumber', 'Date', 'BoosterVersion', 'PayloadMass', 'Orbit',
       'LaunchSite', 'Outcome', 'Flights', 'GridFins', 'Reused', 'Legs',
       'LandingPad', 'Block', 'ReusedCount', 'Serial', 'Longitude',
       'Latitude'],
      dtype='object')


In [13]:
 
import requests
import pandas as pd
# Obtener datos desde la API
url = "https://api.spacexdata.com/v4/launches/past"
response = requests.get(url)
data_json = response.json()
# Convertir a DataFrame
df = pd.json_normalize(data_json)
# Verificar columnas disponibles
print(df.columns)
# Verificar si existe 'LandingPad'
if 'LandingPad' in df.columns:
    print("Valores faltantes en 'LandingPad':", df['LandingPad'].isnull().sum())
else:
    print("La columna 'LandingPad' no existe.")
 

Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')
La columna 'LandingPad' no existe.


In [14]:
 
import requests
import pandas as pd
# Obtener datos desde la API
url = "https://api.spacexdata.com/v4/launches/past"
response = requests.get(url)
data_json = response.json()
# Convertir a DataFrame
df = pd.json_normalize(data_json)
# Extraer 'landing_pad' desde la lista de 'cores'
def extract_landing_pad(cores):
    if isinstance(cores, list) and len(cores) > 0 and isinstance(cores[0], dict):
        return cores[0].get('landing_pad', None)
    return None
df['landingPad'] = df['cores'].apply(extract_landing_pad)
# Ver cuántos valores faltan
print("Valores faltantes en 'landingPad':", df['landingPad'].isnull().sum())

Valores faltantes en 'landingPad': 187


In [15]:
 
import requests
import pandas as pd
# Obtener datos desde la API
url = "https://api.spacexdata.com/v4/launches/past"
response = requests.get(url)
data_json = response.json()
# Convertir a DataFrame
df = pd.json_normalize(data_json)
# Extraer 'landing_pad' desde la lista de 'cores'
def extract_landing_pad(cores):
    if isinstance(cores, list) and len(cores) > 0 and isinstance(cores[0], dict):
        return cores[0].get('landing_pad', None)
    return None
df['landingPad'] = df['cores'].apply(extract_landing_pad)
# Ver cuántos valores faltan
print("Valores faltantes en 'landingPad':", df['landingPad'].isnull().sum())

Valores faltantes en 'landingPad': 187


In [ ]:
 
df['landingPad'] = df['cores'].apply(lambda x: x[0]['landing_pad'] if x and isinstance(x[0], dict) else None)

In [16]:
print(df['landingPad'].isnull().sum())  # Debería dar 26

187


In [18]:
 
def extract_landing_pad(cores):
    if isinstance(cores, list):
        pads = [core.get('landing_pad') for core in cores if isinstance(core, dict)]
        return pads[0] if pads else None
    return None
df['landingPad'] = df['cores'].apply(extract_landing_pad)